In [1]:
from datetime import datetime
from yahooquery import Ticker
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

pd.set_option('display.max_columns', None, 'display.max_rows', 100)

In [7]:
def calc_returns(df, symbol):
    aux = df.loc[df['symbol'] == symbol].reset_index(drop=True)
    aux['prev_close'] = aux['close'].shift(1)
    aux['return'] = (aux['close'] / aux['prev_close'] - 1).fillna(0)
    aux['cum_ret'] = 1
    aux['cum_ret'] += aux['return']
    aux['cum_ret'] = aux['cum_ret'].cumprod()
    return aux

In [3]:
spx = Ticker('^GSPC')

In [8]:
spx_df = spx.history(start=datetime(2018, 1, 1), end=datetime(2023, 1, 1))
spx_df = spx_df.reset_index()
spx_df = calc_returns(spx_df, '^GSPC')

In [9]:
spx_df

,symbol,date,open,high,low,close,volume,adjclose,prev_close,return,cum_ret
0,^GSPC,2018-01-02,2683.729980,2695.889893,2682.360107,2695.810059,3397430000,2695.810059,NaN,0.000000,1.000000
1,^GSPC,2018-01-03,2697.850098,2714.370117,2697.770020,2713.060059,3544030000,2713.060059,2695.810059,0.006399,1.006399
2,^GSPC,2018-01-04,2719.310059,2729.290039,2719.070068,2723.989990,3697340000,2723.989990,2713.060059,0.004029,1.010453
3,^GSPC,2018-01-05,2731.330078,2743.449951,2727.919922,2743.149902,3239280000,2743.149902,2723.989990,0.007034,1.017561
4,^GSPC,2018-01-08,2742.669922,2748.510010,2737.600098,2747.709961,3246160000,2747.709961,2743.149902,0.001662,1.019252
...,...,...,...,...,...,...,...,...,...,...,...
1254,^GSPC,2022-12-23,3815.110107,3845.800049,3797.010010,3844.820068,2819280000,3844.820068,3822.389893,0.005868,1.426221
1255,^GSPC,2022-12-27,3843.340088,3846.649902,3813.219971,3829.250000,3030300000,3829.250000,3844.820068,-0.004050,1.420445
1256,^GSPC,2022-12-28,3829.560059,3848.320068,3780.780029,3783.219971,3083520000,3783.219971,3829.250000,-0.012021,1.403370
1257,^GSPC,2022-12-29,3805.449951,3858.189941,3805.449951,3849.280029,3003680000,3849.280029,3783.219971,0.017461,1.427875


In [13]:
spx_df.to_parquet('data/spx_returns.parquet')

In [10]:
fig = make_subplots(specs=[[{"secondary_y": True}]])


fig.add_trace(
    go.Scatter(x=spx_df['date'], y=spx_df['close'], name="SPX Close"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=spx_df['date'], y=spx_df['cum_ret'], name="SPX Return"),
    secondary_y=True,
)

fig.update_layout(title_text="SPX Return")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Close", secondary_y=False)
fig.update_yaxes(title_text="Return", secondary_y=True)
fig

In [11]:
def stats(df):
    r = {
        'ret': df['cum_ret'].iloc[-1] - 1,
        'mean_ret': df['return'].mean(),
        'vol': df['return'].std()
    }
    r['sharpe'] = round(r['mean_ret'] / r['vol'], 3)
    for i in ('ret', 'mean_ret', 'vol'):
        r[i] = f'{100 * r[i]:.2f}%'

    return r

In [12]:
stats(spx_df)

{'ret': '42.42%', 'mean_ret': '0.04%', 'vol': '1.38%', 'sharpe': 0.027}

---

### Load comp

In [14]:
comp_df = pd.read_parquet('data/spx_comp.parquet')

In [15]:
symbols = list(comp_df.columns)
start = comp_df.index.min()
end = comp_df.index.max()
len(symbols), start, end

(633, datetime.date(2018, 6, 17), datetime.date(2023, 7, 25))

In [16]:
comp_df

ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACGL,ACN,ADBE,ADI,ADM,ADP,ADS,ADSK,AEE,AEP,AES,AET,AFL,AGN,AIG,AIRC,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMG,AMGN,AMP,AMT,AMZN,ANDV,ANET,ANSS,ANTM,AON,AOS,APA,APC,APD,APH,APTV,ARE,ARNC,ATO,ATVI,AVB,AVGO,AVY,AWK,AXON,AXP,AYI,AZO,BA,BAC,BALL,BAX,BBT,BBWI,BBY,BDX,BEN,BF.B,BG,BHF,BHGE,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BMY.RT,BR,BRK.B,BRO,BSX,BWA,BXP,C,CA,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CBS,CCI,CCL,CDAY,CDNS,CDW,CE,CEG,CELG,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA,CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COL,COO,COP,COST,COTY,CPB,CPRI,CPRT,CPT,CRL,CRM,CSCO,CSGP,CSX,CTAS,CTL,CTLT,CTRA,CTSH,CTVA,CTXS,CVET,CVS,CVX,CXO,CZR,D,DAL,DCT,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPS,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DWDP,DXC,DXCM,EA,EBAY,ECA-CA,ECL,ED,EFX,EG,EIX,EL,ELV,EMBC,EMN,EMR,ENPH,EOG,EPAM,EQIX,EQR,EQT,ES,ESRX,ESS,ETFC,ETN,ETR,ETSY,EVHC,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDS,FDX,FE,FFIV,FI,FICO,FIS,FISV,FITB,FL,FLIR,FLR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FSLR,FTI,FTNT,FTRE,FTV,GD,GE,GEHC,GEN,GGP,GILD,GIS,GL,GLW,GM,GNRC,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GT,GWW,HAL,HAS,HBAN,HBI,HCA,HCP,HD,HES,HFC,HIG,HII,HLT,HOG,HOLX,HON,HP,HPE,HPQ,HRB,HRL,HRS,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILMN,INCY,INFO,INTC,INTU,INVH,IP,IPG,IPGP,IQV,IR,IRM,ISRG,IT,ITW,IVZ,J,JBHT,JCI,JEC,JEF,JKHY,JNJ,JNPR,JPM,JWN,K,KDP,KEY,KEYS,KHC,KIM,KLAC,KMB,KMI,KMX,KO,KORS,KR,KSS,KSU,L,LB,LDOS,LEG,LEN,LH,LHX,LIN,LKQ,LLL,LLY,LMT,LNC,LNT,LOW,LRCX,LTHM,LUMN,LUV,LVS,LW,LYB,LYV,M,MA,MAA,MAC,MAR,MAS,MAT,MBC,MCD,MCHP,MCK,MCO,MDLZ,MDT,MET,META,MGM,MHK,MKC,MKTX,MLM,MMC,MMM,MNST,MO,MOH,MOS,MPC,MPWR,MRK,MRNA,MRO,MS,MSCI,MSFT,MSI,MTB,MTCH,MTD,MU,MXIM,MYL,NBL,NCLH,NDAQ,NDSN,NEE,NEM,NFLX,NFX,NI,NKE,NKTR,NLOK,NLSN,NOC,NOV,NOW,NRG,NSC,NTAP,NTRS,NUE,NVDA,NVR,NWL,NWS,NWSA,NXPI,O,ODFL,OGN,OKE,OMC,ON,ORCL,ORLY,OTIS,OXY,OXY.WT,OXY.WTWI,PANW,PARA,PAYC,PAYX,PBCT,PCAR,PCG,PEAK,PEG,PENN,PEP,PFE,PFG,PG,PGR,PH,PHIN,PHM,PKG,PKI,PLD,PM,PNC,PNR,PNW,PODD,POOL,PPG,PPL,PRGO,PRU,PSA,PSX,PTC,PVH,PWR,PX,PXD,PYPL,QCOM,QRVO,RCL,RE,REG,REGN,RF,RHI,RHT,RJF,RL,RMD,ROK,ROL,ROP,ROST,RRC,RSG,RTN,RTX,RVTY,SBAC,SBNY,SBUX,SCG,SCHW,SEDG,SEE,SHW,SIVB,SJM,SLB,SLG,SNA,SNPS,SO,SPG,SPGI,SRCL,SRE,STE,STI,STLD,STT,STX,STZ,SWK,SWKS,SYF,SYK,SYMC,SYY,T,TAP,TDG,TDY,TECH,TEL,TER,TFC,TFX,TGT,TIF,TJX,TMK,TMO,TMUS,TPR,TRGP,TRIP,TRMB,TROW,TRV,TSCO,TSLA,TSN,TSS,TT,TTWO,TWTR,TWX,TXN,TXT,TYL,UA,UAA,UAL,UDR,UHS,ULTA,UNH,UNM,UNP,UPS,URI,USB,UTX,V,VAR,VFC,VIAB,VIAC,VICI,VLO,VMC,VNO,VNT,VRSK,VRSN,VRTX,VSCO,VTR,VTRS,VZ,WAB,WAT,WBA,WBD,WCG,WDC,WEC,WELL,WFC,WHR,WLTW,WM,WMB,WMT,WRB,WRK,WST,WTW,WU,WY,WYNN,XEC,XEL,XL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
dt,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-06-17,0.000896,0.000781,0.000405,0.040851,0.006637,0.000650,0.000784,0.004624,0.000000,0.004231,0.005350,0.001586,0.001082,0.002628,0.000457,0.001274,0.000567,0.001331,0.000358,0.002595,0.001514,0.002423,0.002076,0.0,0.000267,0.000223,0.000496,0.000587,0.000434,0.001106,0.000319,0.001401,0.000324,0.001131,0.002202,0.000000,0.000565,0.000726,0.000370,0.005240,0.000928,0.002571,0.029228,0.000841,0.000000,0.000630,0.002612,0.001475,0.000392,0.000680,0.001543,0.001519,0.001175,0.001158,0.000545,0.000319,0.000000,0.002486,0.000980,0.004646,0.000392,0.000596,0.000000,0.003024,0.000217,0.000790,0.008454,0.011901,0.000000,0.001586,0.001742,0.000000,0.000785,0.002616,0.000457,0.000584,0.